# My Implementation

In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, RandomFlip, RandomRotation, RandomZoom, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.metrics import binary_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix

seed = 415
batch_size = 8
image_path = "./images"
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2,
                              zoom_range = 0.1, # Randomly zoom image
                              width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                              height_shift_range=0.1,
                             rotation_range=30
                             )


# I changed the imagery to grayscale to speed up the training process
train_generator = datagen.flow_from_directory(
    image_path,
    target_size=(227, 227),  # resize for alexnet
    batch_size=batch_size,
    subset='training',
    color_mode="grayscale",
    )

test_generator = datagen.flow_from_directory(
    image_path,
    target_size=(227, 227),  # resize for alexnet
    batch_size=batch_size,
    subset='validation',
    color_mode="grayscale",
    )

train_class_counts = train_generator.classes
test_class_counts = test_generator.classes

train_class_count = dict(zip(train_generator.class_indices.keys(), np.zeros(len(train_generator.class_indices), dtype=int)))
test_class_count = dict(zip(test_generator.class_indices.keys(), np.zeros(len(test_generator.class_indices), dtype=int)))

for label in train_class_counts:
    train_class_count[list(train_generator.class_indices.keys())[int(label)]] += 1

for label in test_class_counts:
    test_class_count[list(test_generator.class_indices.keys())[int(label)]] += 1

print('Number of training samples in each class in the training set:', train_class_count)
print('Number of test samples in each class in the testing set:', test_class_count)








Found 17013 images belonging to 3 classes.
Found 4251 images belonging to 3 classes.
Number of training samples in each class in the training set: {'happy': 7192, 'neutral': 4959, 'sad': 4862}
Number of test samples in each class in the testing set: {'happy': 1797, 'neutral': 1239, 'sad': 1215}


In [3]:

my_callbacks = [
    EarlyStopping(monitor="val_categorical_accuracy", 
                  patience=250,
                  restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_categorical_accuracy", 
                      factor=0.50, patience=150, 
                      verbose=1,
                      min_delta=0.0001),
]



leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.01)
activation = "relu"

# build the model - I resized the images to use AlexNet
# Model architecture
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation=activation, input_shape=(227,227,1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation=activation),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation=activation),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(256, activation=activation),
    Dropout(0.5),
    Dense(128, activation=activation),
    Dropout(0.5),
    Dense(3, activation='softmax')
    ])

model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['categorical_accuracy']
              )

model.build(input_shape=(None, 227, 227, 1))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 225, 225, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 225, 225, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 110, 110, 64)     256       
 hNormalization)                                                 
                                                        

2023-04-13 23:25:46.822331: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-13 23:25:46.822345: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-13 23:25:46.822358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-04-13 23:25:46.822529: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
history = model.fit(train_generator,
                    epochs=500, 
                    validation_data=test_generator,
                    callbacks=my_callbacks)

model.save("alex_net_emotions_local_1.h5")

Epoch 1/500
2127/2127 [==============================] - 455s 213ms/step - loss: 7.0699 - categorical_accuracy: 0.3907 - val_loss: 1.2501 - val_categorical_accuracy: 0.4227 - lr: 0.0010
Epoch 2/500
2127/2127 [==============================] - 441s 207ms/step - loss: 1.1251 - categorical_accuracy: 0.4194 - val_loss: 1.0814 - val_categorical_accuracy: 0.4227 - lr: 0.0010
Epoch 3/500
2127/2127 [==============================] - 433s 204ms/step - loss: 1.1100 - categorical_accuracy: 0.4225 - val_loss: 1.0813 - val_categorical_accuracy: 0.4227 - lr: 0.0010
Epoch 4/500
2127/2127 [==============================] - 433s 204ms/step - loss: 1.0939 - categorical_accuracy: 0.4224 - val_loss: 1.0813 - val_categorical_accuracy: 0.4227 - lr: 0.0010
Epoch 5/500
2127/2127 [==============================] - 431s 203ms/step - loss: 1.1045 - categorical_accuracy: 0.4226 - val_loss: 1.0813 - val_categorical_accuracy: 0.4227 - lr: 0.0010
Epoch 6/500
2127/2127 [==============================] - 431s 202ms/st

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Make predictions on the test set
y_pred = model.predict(test_generator)
y_actual = test_generator.classes
y_pred = np.argmax(y_pred,axis=1)


y_pred = np.round(y_pred)

confusion_mtx = confusion_matrix(y_actual, y_pred)
print(confusion_mtx)


In [ ]:
# Evaluation
print(classification_report(test_generator.classes, y_pred))

plt.imshow(confusion_mtx, cmap='binary', interpolation='nearest')
plt.colorbar()

tick_marks = np.arange(3)
plt.xticks(tick_marks, ['Happy', 'Neutral', 'Sad'], rotation=45)
plt.yticks(tick_marks, ['Happy', 'Neutral', 'Sad'])

thresh = confusion_mtx.max() / 2.
for i in range(confusion_mtx.shape[0]):
    for j in range(confusion_mtx.shape[1]):
        plt.text(j, i, format(confusion_mtx[i, j]), ha="center", va="center", color="white" if confusion_mtx[i, j] > thresh else "black")

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('CNN Model')

plt.show()

plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()